# Exercise (with one solution)

From Lab session **4.2** and **4.3**, we know the process for training a classifier based on the neural networks.
Now, let's build your own network according to the **4.2** and **4.3**.


We use the MNIST dataset to train the networks.

# Step 1: Import the libraries



In [ ]:
#import  the libraries
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F


# Step 2: Load the dataset and visualize the dataset
We can load the MNIST dataset from ``torchvision`` with the following parameters:

 - **root** is the path where the train/test data is stored,
 - **train** specifies training or test dataset,
 - **download=True** downloads the data from the internet if it’s not available at root.
 - **transform** and **target_transform** specify the feature and label transformations


Besides, we can index Datasets manually like a list: train_set[index]. We use matplotlib to visualize some samples in our training data.


In [ ]:
# Load MNIST dataset

train_set = torchvision.datasets.MNIST(root = './data/MNIST', download = True,
                                              train = True, transform = transforms.Compose([transforms.ToTensor(),]))

test_set = torchvision.datasets.MNIST(root = './data/MNIST', download=True,
                                             train=False, transform = transforms.Compose([transforms.ToTensor()]))


# Visulize some figures
figure = plt.figure(figsize=(10, 10))
cols, rows = 6, 6
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_set), size=(1,)).item()
    img, label = train_set[sample_idx]
    figure.add_subplot(rows, cols, i)
    # plt.title("Label "+ str(label))
    plt.title("Ground Truth:{}".format(label))
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")

plt.show()

Preparing your data for training with DataLoaders




In [ ]:
# DataLoader with the batch_size
print("Training data size: {}".format(len(train_set)))
train_size=len(train_set)
test_size=len(test_set)
print("Training data size: {}".format(len(test_set)))

train_loader = torch.utils.data.DataLoader(train_set,batch_size=20)

test_loader = torch.utils.data.DataLoader(test_set,batch_size=10000)

# Step 3: Define your own network model

Here, you can design any network model to classify the dataset. You can add any layers.



In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# define the model
model = NeuralNetwork()

print(model)

# 4. Define Loss and Optimizer

Define the loss and the optimizer and their hyper-parameters, such as number of epochs, and learning rate.




In [ ]:
#  you can set epoch size

# define learning rate
learning_rate = 0.005
# define your optimizer with SGD and learning rate
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
# define the loss function
criterion = nn.CrossEntropyLoss()
# set the epoch
epochs = 15

# Step 5: Train your own network model

You can set up parameters.


In [ ]:
# loop over the dataset multiple times
losses = []

for i in range(epochs):
  for j,(images,targets) in enumerate(train_loader):

    #making predictions
    y_pred = model(images)

    #calculating loss
    loss = criterion(y_pred,targets.reshape(-1))
    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  if i>10:
    optimizer.lr = 0.0005
  print(loss)
  losses.append(loss)

print('Complete Training')


# Step 6: Evaluate the result on the test dataset and analyze the performance on the whole dataset

You can show the best performance with any update mthods.




In [ ]:
# Test the result on test data

# make a prediction for a sample randomly chose from the test dataset
# you can run this part repeatedly, see the classification result from the model
import random
rand_no = random.randint(0,10000)
print(rand_no)

x_test, y_test = next(iter(test_loader))

y_pred = (model(x_test).argmax(dim=1))

plt.imshow(x_test[rand_no].reshape(28,28),cmap='gray')

pred = model(x_test[rand_no].reshape(-1,1,28,28)).argmax()

print("Prediction is {}".format(pred))

# compute the accuracy of the model

print("Accuracy is : ",(y_pred.eq(y_test).sum()/test_size).item()*100,"%")

This is just another way of visualizing the dataset.

In [ ]:
# Visulize some test figures
figure = plt.figure(figsize=(10, 10))
cols, rows = 6, 5
for i in range(1, cols * rows + 1):   # for loop to get multiple images to predict (instead of only one)

    # getting one image to predict
    sample_idx = torch.randint(len(test_set), size=(1,)).item()
    print("Prediction: {}".format(sample_idx))

    # predict
    img, label = test_set[sample_idx]
    pred = model(img.reshape(-1,1,28,28)).argmax()

    # plot
    figure.add_subplot(rows, cols, i)
    plt.title("Prediction: {}".format(pred))
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

Finally, this is another way of processing the test set and computing the metrics.

In fact, this is the way we usually do.

In [ ]:
# we also can plot the Confusion Matrix
import pandas as pd
import torch.nn.functional as F

# accuracy function
def testing_accuracy(model, data_loader):
    model.eval()  # IMPORTANT: we use this to prevent the network from learning using the test set
    test_loss = 0
    device = 'cpu'

    y_pred = []
    y_actu = []
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)  # predict
            test_loss += criterion(output, target)  # loss - same loss used during training

            # get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            y_pred.extend(torch.flatten(pred).tolist())
            y_actu.extend(target.tolist())

    y_pred = pd.Series(y_pred, name='Actual')
    y_actu = pd.Series(y_actu, name='Predicted')
    cm = pd.crosstab(y_actu, y_pred)
    correct = sum([cm.iloc[i,i] for i in range(len(cm))])

    test_loss /= len(data_loader.dataset)
    accuracy = 100*correct/len(test_loader.dataset)

    return(test_loss, accuracy, cm)

test_results = testing_accuracy(model, test_loader)

print("- Test Loss: ", test_results[0], "\n")
print("- Accuracy: ", test_results[1], "\n")
print("- Confusion Matrix: \n \n",  test_results[2] )

### Creating Heatmap for Confusion Matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

conf_matrix = test_results[2].to_numpy()

fig, ax = plt.subplots(figsize=(10,6))
im = ax.imshow(conf_matrix)

ax.set_xticks(np.arange(10))
ax.set_yticks(np.arange(10))

for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        text = ax.text(j, i, conf_matrix[i, j],
                       ha="center", va="center", color="w")

ax.set_xlabel('Actual Labels')
ax.set_ylabel('Predicted Labels')
ax.set_title('Confusion Matrix')

# Step 7: Any other thought on the training network?
 - How to save the training file?
 - How to speed the training process?


In [ ]:
#  train on the GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)


# save the training model
PATH = './mnist_net.pth'
torch.save(model.state_dict(), PATH)

